In [10]:
# Import all the libraries and dependencies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Reading the data and preprocessing it
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/C.V/data/Train'
test_dir = '/content/drive/MyDrive/C.V/data/Test'

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
# Image preprocessing
train_generator = train_datagen.flow_from_directory(train_dir, # path for training set
                                                    target_size=(150, 150), # setting input size to ensure uniformity
                                                    batch_size=32, # creating batches of dataset
                                                    class_mode='categorical') #categorical as we have 3 categories
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 385 images belonging to 3 classes.
Found 138 images belonging to 3 classes.


In [14]:
# importing the VGG16 model from tensorflow and using the same weights applied during imagenet competition
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
# this will exclude the top layers from training phase as our dataset only has 3 categories
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [15]:
# Model creation
model = Sequential()
model.add(vgg16) # adding VG16 model architecture
model.add(Flatten()) # to flatten and create a single dimensional array
model.add(Dense(128, activation="relu")) 
model.add(Dense(64, activation="relu"))
model.add(Dense(3, activation='softmax')) 
#adding the output layer with softmax function as this is a multi label classification problem.
vgg16.trainable = False
# this will exclude the initial layers from training phase as there are already been trained.

In [16]:
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit_generator(train_generator,epochs=5) # fitting the model on the data

Epoch 1/5


<ipython-input-16-fb2c6d3c7ab5>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=5)


13/13 [==============================] - 2s 108ms/step - loss: 0.8122 - accuracy: 0.6701
Epoch 2/5
13/13 [==============================] - 2s 112ms/step - loss: 0.3556 - accuracy: 0.8753
Epoch 3/5
13/13 [==============================] - 2s 116ms/step - loss: 0.2109 - accuracy: 0.9221
Epoch 4/5
13/13 [==============================] - 1s 120ms/step - loss: 0.0893 - accuracy: 0.9740
Epoch 5/5
13/13 [==============================] - 2s 112ms/step - loss: 0.0392 - accuracy: 0.9974


In [38]:
# Model evaluation
test_loss, test_accuracy = model.evaluate(test_generator)
print("The testing datas loss percentage calculated is:",test_loss)
print("The accuracy of the while testing the model is: ",test_accuracy)

5/5 [==============================] - 1s 84ms/step - loss: 0.4659 - accuracy: 0.8406
The testing datas loss percentage calculated is: 0.46591782569885254
The accuracy of the while testing the model is:  0.8405796885490417


In [43]:
#Test the model

from tensorflow.keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

img = load_img('/content/drive/MyDrive/C.V/data/b1.jpg',target_size=(150, 150) )
x = np.array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds=model.predict(x)
# create a list containing the class labels
class_labels = ['Apple','Banana','Orange']

# find the index of the class with maximum score
pred = np.argmax(preds, axis=-1)

# print the label of the class with maximum score
print(class_labels[pred[0]])

1/1 [==============================] - 0s 20ms/step
Banana
